In [ ]:
import numpy as np

class MLP:
    def __init__(self, layer_sizes, learning_rate=0.01):
        """
        layer_sizes örn: [2, 8, 6, 1]
        """
        self.layer_sizes = layer_sizes
        self.lr = learning_rate
        self.L = len(layer_sizes) - 1  # weight layer sayısı
        
        self.weights = []
        self.biases = []
        
        # He initialization (ReLU için uygun)
        for i in range(self.L):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i])
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)

    # ------------------
    # Aktivasyonlar
    # ------------------

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, a):
        return a * (1 - a)

    # ------------------
    # Forward Pass
    # ------------------

    def forward(self, X):
        self.activations = [X]
        self.z_values = []
        
        A = X
        
        for i in range(self.L):
            Z = A @ self.weights[i] + self.biases[i]
            self.z_values.append(Z)
            
            if i == self.L - 1:
                A = self.sigmoid(Z)  # output layer
            else:
                A = self.relu(Z)
                
            self.activations.append(A)
            
        return A

    # ------------------
    # Loss
    # ------------------

    def compute_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    # ------------------
    # Backpropagation
    # ------------------

    def backward(self, y_true):
        m = y_true.shape[0]
        
        dA = -(y_true - self.activations[-1])
        
        for i in reversed(range(self.L)):
            A_prev = self.activations[i]    # bu katmana giriş
            Z = self.z_values[i]            # bu katmanın lineer çıktısı
            
            if i == self.L - 1:             # aktivasyon fonksiyonunun türevini uygula
                dZ = dA * self.sigmoid_derivative(self.activations[-1])
            else:
                dZ = dA * self.relu_derivative(Z)
            
            dW = (A_prev.T @ dZ) / m                             # Ağırlık gradyanını hesapla → dW (Her ağırlığın hataya ne kadar katkıda bulunduğunu ölçer.)
            db = np.sum(dZ, axis=0, keepdims=True) / m           # Bias gradyanını hesapla → db (Biaslerin hataya ne kadar katkıda bulunduğunu ölçer.)
            
            dA = dZ @ self.weights[i].T                          # Bir önceki katmana gradyanı aktar → dA (Bir sonraki katmanın hatasının, bu katmanın ağırlıklarıyla çarpılmasıyla elde edilir.) Bir sonraki iterasyonda bu dA kullanılacak.
            
            self.weights[i] -= self.lr * dW                     # Ağırlıkları güncelle
            self.biases[i] -= self.lr * db                      # Biasleri güncelle self.lr = öğrenme oranı (Ağırlıklar, gradyanın tersi yönünde küçük adımlarla güncellenir.)

    # ------------------
    # Training
    # ------------------

    def train(self, X, y, epochs=5000, verbose=True):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            self.backward(y)
            
            if verbose and epoch % 500 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.6f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred).astype(float)

In [2]:
# XOR dataset
X = np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
])

y = np.array([
    [0],
    [1],
    [1],
    [0]
])

In [17]:

# 3 gizli katmanlı ağ
model = MLP([2, 8, 4, 2, 1], learning_rate=0.1)

model.train(X, y, epochs=10000)

print("\nTahminler:")
print(model.predict(X))

Epoch 0, Loss: 0.270130
Epoch 500, Loss: 0.140316
Epoch 1000, Loss: 0.073015
Epoch 1500, Loss: 0.031617
Epoch 2000, Loss: 0.016176
Epoch 2500, Loss: 0.010001
Epoch 3000, Loss: 0.006967
Epoch 3500, Loss: 0.005238
Epoch 4000, Loss: 0.004155
Epoch 4500, Loss: 0.003414
Epoch 5000, Loss: 0.002886
Epoch 5500, Loss: 0.002487
Epoch 6000, Loss: 0.002181
Epoch 6500, Loss: 0.001938
Epoch 7000, Loss: 0.001742
Epoch 7500, Loss: 0.001579
Epoch 8000, Loss: 0.001443
Epoch 8500, Loss: 0.001327
Epoch 9000, Loss: 0.001227
Epoch 9500, Loss: 0.001141

Tahminler:
[[0.05893996]
 [0.97473793]
 [0.99563534]
 [0.01144595]]
